In [1]:
import socket
import time
import os
import sys
import platform
from pathlib import Path
import datetime as datetime
import subprocess
import re

module_path = os.path.abspath(os.path.join('./wpshelper'))
if module_path not in sys.path:
    sys.path.append(module_path)
from wpshelper import wps_open, wps_configure, wps_start_record, wps_stop_record, wps_analyze_capture, wps_save_capture, wps_export_html, wps_close

# run the remote commands on a Linux machine
def run_command(command):
    if command is None:
        command = "ls" # default command 

    # Run the command and capture the output
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True)
    stdout, stderr = process.communicate()
    stdout_str = str(stdout)
    # Check for errors
    if process.returncode != 0:
        log_entry = f"run_command: An error occurred while executing the command {command}:\n{stderr}"
    else:
        log_entry = f"run_command: Command output:\n{stdout}"
    print(log_entry)
    return stdout_str

ImportError: cannot import name 'wps_export_html_json' from 'wpshelper' (/home/user/bluetooth/wps/wpshelper/wpshelper/__init__.py)

In [ ]:

# =======================START OF CONFIGURATION SECTION (USER MUST MODIFY TO MATCH THEIR SETTINGS)================
# See the manual Automation Server Protocol.pdf for more details
TCP_IP = '192.168.147.1'    # The IP address of the interface that the automation server is running on
TCP_PORT = 22901        # The default port to talk with the automation server

MAX_TO_READ = 1000

# This will capture Bluetooth LE
# This is described in the section "IOParameters – Sodera and X500 of the Automation Server Protocol manual
# IMPORTANT: This is ignored for the X240. For configurations that include one or more X240 devices, the capture 
# technology should be configured by using the appropriate datasource prior to starting capture via automation. 
# Configuration of capture technology and initiating firmware update are not supported via the automation interface. 
capture_technology = "capturetechnology=bredr-off|le-on|2m-on|spectrum-off|wifi-off"

# Only one personality_key should be uncommented and it should be the device that is connected and powered up before
# running the script. Additional personality keys to support other equipment and configurations 
# can be found under the section "Start FTS" in the Automation Server Protocol manual.
#
# Only uncomment one of the personality_key statements below:
personality_key = "SODERA"
# personality_key = "X240"  
# personality_key = "X500"

# Setup the path to the the location of FTS.exe. Change this to your directory
wps_path=r'C:\Program Files (x86)\Teledyne LeCroy Wireless\Wireless Protocol Suite 2.60'


# This is the directory where the capture folder will be created. Make sure this path exists 
data_path=r'C:\Users\Public\Documents\share\input'

# This will be the name of the capture folder
capture_name = 'le_capture_' + time.strftime("%Y%m%d_%H%M%S")

# ===================================END OF CONFIGURATION SECTION=====================================

# Automatically derive the following data
personality_key = personality_key.strip()  # remove any spaces

capture_absolute_filename =  data_path + '\\' +  capture_name + ".cfa" 
html_absolute_filename =  data_path + '\\' +  capture_name + ".html" 

wps_executable_path = wps_path + '\\' +  'Executables\\Core'


In [5]:
wps_handle = wps_open(tcp_ip=TCP_IP,tcp_port=TCP_PORT,max_to_read = 1000,wps_executable_path=wps_executable_path,personality_key=personality_key)

In [6]:
wps_configure(wps_handle, personality_key,capture_technology)
wps_start_record(wps_handle)


In [7]:
wps_stop_record(wps_handle)
wps_analyze_capture(wps_handle)
wps_save_capture(wps_handle, capture_absolute_filename)

In [9]:
wps_close(wps_handle)

In [8]:
print('\n'.join(wps_handle['log']))

wps_open: Trying connection. Receiving: b'TCP CONNECT;SUCCEEDED;Timestamp=7/9/2023 10:19:51 AM;Reason=C:\\Program Files (x86)\\Teledyne LeCroy Wireless\\Wireless Protocol Suite 2.60\\Executables\\Core\\\r\n'
wps_open: s1 Sending: b'Start FTS;C:\\Program Files (x86)\\Teledyne LeCroy Wireless\\Wireless Protocol Suite 2.60\\Executables\\Core;SODERA'
wps_open: s1 received: IS INITIALIZED;SUCCEEDED;Timestamp=7/9/2023 10:20:11 AM;Reason=yes

wps_open: Received data parsed to ['IS INITIALIZED', 'SUCCEEDED', 'Timestamp=7/9/2023 10:20:11 AM', 'Reason=yes\r\n'], which indicates startup is not complete. Still waiting for the command START FTS with a status of SUCCEEDED.
wps_open: s1 received: IS INITIALIZED;SUCCEEDED;Timestamp=7/9/2023 10:20:14 AM;Reason=yes

wps_open: Received data parsed to ['IS INITIALIZED', 'SUCCEEDED', 'Timestamp=7/9/2023 10:20:14 AM', 'Reason=yes\r\n'], which indicates startup is not complete. Still waiting for the command START FTS with a status of SUCCEEDED.
wps_open: s1 

In [25]:
# Add a Bookmark
bookmark_text = 'le collect'
bookmark_frame = 1
def wps_save_capture(handle, bookmark_frame, bookmark_text):
    s = handle['socket']
    MAX_TO_READ = handle['max_data_from_automation_server']
    
    FTE_CMD=f"Add Bookmark;string={bookmark_text};frame={bookmark_frame}"
    send_data=FTE_CMD.encode(encoding='UTF-8',errors='strict')
    log_entry = f"wps_save_capture: Sending: {send_data}"
    handle['log'].append(log_entry)
    
    s.send(send_data)
    data=s.recv(MAX_TO_READ)
    log_entry = f"wps_save_capture: {data}"
    handle['log'].append(log_entry)

Sending: b'Add Bookmark;string=le collect;frame=1'
b'ADD BOOKMARK;SUCCEEDED;FRAME=1;Timestamp=6/25/2023 7:08:33 PM\r\n'
